# DEAP

DEAP is a novel evolutionary computation framework for rapid prototyping and testing of ideas. It seeks to make algorithms explicit and data structures transparent. It works in perfect harmony with parallelisation mechanism such as multiprocessing and SCOOP. The following documentation presents the key concepts and many features to build your own evolutions.

Library documentation: <a>http://deap.readthedocs.org/en/master/</a>

## One Max Problem (GA)

This problem is very simple, we search for a 1 filled list individual. This problem is widely used in the evolutionary computation community since it is very simple and it illustrates well the potential of evolutionary algorithms.

In [1]:
import random

from deap import base
from deap import creator
from deap import tools

In [2]:
# creator is a class factory that can build new classes at run-time
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [3]:
# a toolbox stores functions and their arguments
toolbox = base.Toolbox()

# attribute generator
toolbox.register("attr_bool", random.randint, 0, 1)

# structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [4]:
# evaluation function
def evalOneMax(individual):
    return sum(individual),

In [5]:
# register the required genetic operators
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [6]:
random.seed(64)

# instantiate a population
pop = toolbox.population(n=300)
CXPB, MUTPB, NGEN = 0.5, 0.2, 40

# evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

print("  Evaluated %i individuals" % len(pop))

  Evaluated 300 individuals


In [7]:
# begin the evolution
for g in range(NGEN):
    print("-- Generation %i --" % g)

    # select the next generation individuals
    offspring = toolbox.select(pop, len(pop))

    # clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(invalid_ind))

    # the population is entirely replaced by the offspring
    pop[:] = offspring

    # gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)

-- Generation 0 --
  Evaluated 189 individuals
  Min 40.0
  Max 65.0
  Avg 54.7433333333
  Std 4.46289766358
-- Generation 1 --
  Evaluated 171 individuals
  Min 44.0
  Max 70.0
  Avg 58.48
  Std 3.98533980149
-- Generation 2 --
  Evaluated 169 individuals
  Min 54.0
  Max 68.0
  Avg 61.6066666667
  Std 2.92779021714
-- Generation 3 --
  Evaluated 185 individuals
  Min 57.0
  Max 73.0
  Avg 63.82
  Std 2.74364720764
-- Generation 4 --
  Evaluated 175 individuals
  Min 54.0
  Max 73.0
  Avg 65.67
  Std 2.57961883489
-- Generation 5 --
  Evaluated 164 individuals
  Min 60.0
  Max 76.0
  Avg 67.5466666667
  Std 2.57833710407
-- Generation 6 --
  Evaluated 185 individuals
  Min 63.0
  Max 77.0
  Avg 69.0666666667
  Std 2.50510589707
-- Generation 7 --
  Evaluated 194 individuals
  Min 62.0
  Max 78.0
  Avg 70.78
  Std 2.39963886172
-- Generation 8 --
  Evaluated 199 individuals
  Min 63.0
  Max 79.0
  Avg 72.3133333333
  Std 2.57717330077
-- Generation 9 --
  Evaluated 169 individuals
  Mi

In [8]:
best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

Best individual is [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], (100.0,)


## Symbolic Regression (GP)

Symbolic regression is one of the best known problems in GP. It is commonly used as a tuning problem for new algorithms, but is also widely used with real-life distributions, where other regression methods may not work.

All symbolic regression problems use an arbitrary data distribution, and try to fit the most accurately the data with a symbolic formula. Usually, a measure like the RMSE (Root Mean Square Error) is used to measure an individual’s fitness.

In this example, we use a classical distribution, the quartic polynomial (x^4 + x^3 + x^2 + x), a one-dimension distribution. 20 equidistant points are generated in the range [-1, 1], and are used to evaluate the fitness.

In [9]:
import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# define a new function for divison that guards against divide by 0
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

In [10]:
# add aritmetic primitives
pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

# constant terminal
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

# define number of inputs
pset.renameArguments(ARG0='x')

In [11]:
# create fitness and individual objects
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [12]:
# register evolution process parameters through the toolbox
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

# evaluation function
def evalSymbReg(individual, points):
    # transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# prevent functions from getting too deep/complex
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [13]:
# compute some statistics about the population
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

In [14]:
random.seed(318)

pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)

# run the algorithm
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                               halloffame=hof, verbose=True)

   	      	                fitness                	              size             
   	      	---------------------------------------	-------------------------------
gen	nevals	avg    	max    	min     	std    	avg    	max	min	std    
0  	300   	2.39949	59.2593	0.165572	4.64122	3.69667	7  	2  	1.61389
1  	146   	1.0971 	10.1   	0.165572	0.845978	3.80667	13 	1  	1.78586
2  	169   	0.902365	6.5179 	0.165572	0.72362 	4.16   	13 	1  	2.0366 
3  	167   	0.852725	9.6327 	0.165572	0.869381	4.63667	13 	1  	2.20408
4  	158   	0.74829 	14.1573	0.165572	1.01281 	4.88333	13 	1  	2.14392
5  	160   	0.630299	7.90605	0.165572	0.904373	5.52333	14 	1  	2.09351
6  	181   	0.495118	4.09456	0.165572	0.524658	6.08333	13 	1  	1.99409
7  	170   	0.403873	2.6434 	0.165572	0.440596	6.34667	14 	1  	1.84386
8  	173   	0.393405	2.9829 	0.165572	0.425415	6.37   	12 	1  	1.78132
9  	168   	0.414299	13.5996	0.165572	0.841226	6.25333	11 	2  	1.76328
10 	142   	0.384179	4.07808	0.165572	0.477269	6.25667	13 	1  	1.78067